In [1]:
import json
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

IMG_SIZE = 224

with open("labels.json", "r") as f:
    meta = json.load(f)

LABELS = meta["classes"]

model = tf.keras.models.load_model("vgg_FT_best.keras")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def predict_rgb_face(face_rgb):
    face_rgb_resized = cv2.resize(face_rgb, (IMG_SIZE, IMG_SIZE))
    face_rgb_resized = np.expand_dims(face_rgb_resized, axis=0)
    face_rgb_resized = tf.keras.applications.vgg16.preprocess_input(face_rgb_resized)
    
    probs = model.predict(face_rgb_resized, verbose=0)[0]
    idx = int(np.argmax(probs))
    
    return LABELS[idx], float(probs[idx])

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        face_bgr = frame[y:y+h, x:x+w]
        face_rgb = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB)
        
        name, conf = predict_rgb_face(face_rgb)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"{name} ({conf:.2f})", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    cv2.imshow("Face Recognition", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import time

test_model = tf.keras.models.load_model("vgg_FT_best.keras")

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Camera is not detected.")

def preprocess(frame, img_size):
    img = cv2.resize(frame, (img_size, img_size))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

while True:
    ret, frame = cap.read()
    if not ret:
        break

    t0 = time.time()

    input_tensor = preprocess(frame, IMG_SIZE)

    # Model Inference
    t1 = time.time()
    preds = test_model(input_tensor, training=False)
    t2 = time.time()

    preprocess_ms = (t1 - t0) * 1000
    inference_ms  = (t2 - t1) * 1000
    total_ms      = (t2 - t0) * 1000

    fps = 1000 / total_ms

    # FPS Display
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Real-Time Evaluation", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
